In [1]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon

In [ ]:
input_directory = "D:\\nysa\\"
output_directory = "D:\\nysa\\nys"

### Zbiornik Nysa

In [ ]:
polygon_geojson = {
    "type": "Polygon",
    "coordinates": [[[17.21957060095079, 50.460772633490535],[17.2134235260186, 50.457448087796706],[17.213177974582408, 50.45252341775182],
                     [17.217720680466954, 50.44486180095984],[17.219439540522302, 50.441734253881094],[17.237978673977125, 50.433992686039176],
                     [17.252957311602472, 50.43055158280865],[17.2549217230941, 50.43618234878534],[17.267199294918214, 50.43274140471141],
                     [17.273829183764633, 50.43649515164927],[17.28291458691419, 50.43657335150047],[17.2853701012792, 50.43328884652985],
                     [17.298138775976724, 50.43493112749999],[17.308943039181628, 50.4589332073684],[17.307224179126166, 50.465576582422074],
                     [17.299857633706523, 50.47026546060073],[17.285983977544646, 50.47456319158093],[17.28033629450593, 50.474250642501005],
                     [17.275179714339686, 50.46878069910437],[17.271741994228847, 50.46557658122478],[17.268918152709034, 50.46385720895421],
                     [17.26044662815056, 50.46565473302471],[17.2557811508577, 50.46370089928391],[17.25062457069143, 50.46377905418362],
                     [17.24755517773542, 50.46151250965917],[17.238224223148734, 50.46104355585072],[17.236505362837676, 50.45893320623739],
                     [17.231348782671404, 50.461512509725054],[17.21957060095079, 50.460772633490535]]]}

polygon = Polygon(polygon_geojson["coordinates"][0])

### Zbiornik Otmuchów

In [ ]:
polygon_geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [17.08263771821214, 50.48507798424535],
            [17.07443826328685, 50.46263894720471],
            [17.087557391166456, 50.452720630917526],
            [17.105596192001798, 50.44763016117196],
            [17.121585129105114, 50.446324824300575],
            [17.15048820865755, 50.45206803655148],
            [17.157457745342327, 50.46629253885732],
            [17.149873249538047, 50.47399025575277],
            [17.135319217045776, 50.474773004216274],
            [17.128554666733123, 50.47973011018317],
            [17.08263771821214, 50.48507798424535]
        ]
    ]
}

polygon = Polygon(polygon_geojson["coordinates"][0])

### Zbiornik Racibórz Dolny

In [ ]:
polygon_geojson = {
    "type": "Polygon",
    "coordinates": [
        [
            [18.21375643828307, 50.06655673400155],
            [18.20970854712627, 50.05989805005862],
            [18.21097351311252, 50.055837423287386],
            [18.27320983965643, 49.98789282538519],
            [18.27953466958965, 49.98317540065318],
            [18.294967254626584, 49.992284494598096],
            [18.285353513127774, 50.005457096167106],
            [18.294714261429533, 50.03097889943476],
            [18.286871472312043, 50.04446583995531],
            [18.284088547141494, 50.05275111709224],
            [18.272450860064254, 50.06038530217677],
            [18.26688500972324, 50.066069544566574],
            [18.265620043735993, 50.069479766691245],
            [18.21375643828307, 50.06655673400155]
        ]
    ]
}

polygon = Polygon(polygon_geojson["coordinates"][0])

In [ ]:
input_files = os.listdir(input_directory)

file_counter = 1

for input_file_name in input_files:
    if input_file_name.endswith(".csv"):
        input_file_path = os.path.join(input_directory, input_file_name)
        a = pd.read_csv(input_file_path)
        
        a['geometry'] = a.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
        filtered_points = a[a['geometry'].apply(lambda point: point.within(polygon))]
        
        filtered_points = filtered_points.drop(columns=['geometry'])
        
        output_file_name = f"file_rac_{file_counter}.csv" 
        output_file_path = os.path.join(output_directory, output_file_name)
        filtered_points.to_csv(output_file_path, index=False)
    
        file_counter += 1

print("Przetworzono wszystkie pliki.")

In [ ]:
csv_files = [f for f in os.listdir(output_directory) if f.endswith('.csv')]

for i, csv_file in enumerate(csv_files):
    file_path = os.path.join(output_directory, csv_file)
    f1 = pd.read_csv(file_path, index_col=0)

    f1 = f1.drop(columns=['mp_type', 'latitude', 'longitude', 'easting', 'northing', 'height',
                          'height_wgs84','line', 'pixel', 'rmse', 'temporal_coherence', 'amplitude_dispersion',
                          'incidence_angle', 'track_angle','los_east', 'los_north', 'los_up', 'mean_velocity',
                          'mean_velocity_std', 'acceleration','acceleration_std', 'seasonality', 
                          'seasonality_std'], inplace=False)
    
    if 'pid' in f1.columns:
        f1.rename(columns={'pid': 'Date'}, inplace=True)

    f1 = f1.transpose()
    
    f1.index = f1.index.map(lambda x: pd.to_datetime(x, format='%Y%m%d', errors='ignore').strftime('%Y-%m-%d') 
                            if not isinstance(x, str) or x.isdigit() else x)
    
    output_file_path = os.path.join(output_directory, f'input_rac_{i+1}.csv') # Tutaj można zmienić nazwę pliku
    f1.to_csv(output_file_path, index=True, header=True)

print("Przetworzono i zapisano pliki CSV.")

In [ ]:
csv_files = [f for f in os.listdir(output_directory) if f.endswith('.csv')]

for i, csv_file in enumerate(csv_files):
    file_path = os.path.join(output_directory, csv_file)
    f1 = pd.read_csv(file_path)

    f1 = f1[['pid', 'latitude', 'longitude', 'height']]
    
    output_file_path = os.path.join(output_directory, f'geo_rac{i+1}.csv') # Tutaj można zmienić nazwę pliku
    f1.to_csv(output_file_path, index=False)

print("Przetworzono i zapisano pliki CSV.")